# Aula 02 - Atributos privados e métodos de acesso

Na primeira aula, um dos princípios mais importantes que citamos na programação orientada a objeto foi o encapsulamento. Segundo este princípio, uma classe deveria ser a única responsável por manipular seus próprios atributos.

Mas considere por um minuto as classes que fizemos até agora. Sempre foi possível acessar e manipular diretamente seus atributos:

In [ ]:
import random

class Usuario:
    # Método construtor
    def __init__(self, nome, cpf, email):
        self.nome = nome
        self.cpf = cpf
        self.login = email
        self.senha = str(random.randint(100000, 999999))
    
    def fazer_login(self, login, senha):
        if login == self.login and senha == self.senha:
            print(self.nome, 'logado com sucesso!')
        else:
            print('Erro! Login ou senha incorretos!')
            
user01 = Usuario('Rafael', 13579024681, 'rafael@letscode.com')

print(user01.nome) # note que estamos acessando diretamente o atributo aqui
user01.senha = '123456' # note que estamos alterando um atributo diretamente
print(user01.senha)
user01.fazer_login('rafael@letscode.com', '123456')

Rafael
123456
Rafael logado com sucesso!


Isso não ocorre apenas no programa principal. Tecnicamente, é possível que objetos de outras classes também manipulem os atributos de objetos de nossa classe. Considere os exercícios 04 e 05 da primeira aula (classes Televisor e ControleRemoto). Era perfeitamente possível implementá-los da seguinte maneira:

In [ ]:
class Televisor:
    def __init__(self, marca, modelo, volume, canal):
        self.marca = marca
        self.modelo = modelo
        self.volume = volume
        self.canal = canal
    ...
    def aumentar_volume(self):
        if self.volume < 100:
            self.volume += 1
        else:
            print('Volume já está no máximo!')
    ...
    
class ControleRemoto:
    def __init__(self, tv):
        self.tv = tv
    ...
    def aumentar_volume(self):
        self.tv.volume += 1

televisor = Televisor('Samsung', 'UHD55SMART', 98, 'HBO')
controle = ControleRemoto(televisor)

# Aumentando o volume via tv:
televisor.aumentar_volume() # funciona, foi pra 99
print(televisor.volume)
televisor.aumentar_volume() # funciona, foi pra 100
print(televisor.volume)
televisor.aumentar_volume() # não funciona, fica em 100
print(televisor.volume)

# Aumentando via controle:
controle.aumentar_volume() # funciona, foi pra 101
print(televisor.volume)

99
100
Volume já está no máximo!
100
101


Ao projetar a classe Televisor, previmos uma regra: volume não deveria passar de 100. Quando acionamos o método de volume do próprio Televisor, ele faz a checagem para evitar um valor inválido.

Porém, como o atributo é livremente acessível, outras classes podem alterá-lo sem passar pelo método. É o que ocorreu na classe ControleRemoto: o programador, por distração ou desconhecimento, alterou diretamente um atributo da classe Televisor dentro da classe ControleRemoto sem fazer qualquer tipo de verificação. Com isso, o ControleRemoto criou uma brecha para que tivéssemos objetos Televisor com volume superior a 100.

Isso é conhecido como _furar o encapsulamento_ da classe Televisor, e geralmente é ruim.

## Níveis de acesso dos atributos

Diversas linguagens orientadas a objeto oferecem uma ferramenta para ajudar a proteger o encapsulamento da classe: restringir o acesso aos atributos da mesma. Na maioria dessas linguagens teremos 3 níveis de acesso, e seu significado varia pouco de linguagem para linguagem. Eles tipicamente são:

* **Private (privado)**: apenas objetos da própria classe possuem acesso ao atributo.
* **Protected (protegido)**: apenas objeto da própria classe ou de classes herdeiras possuem acesso ao atributo.
* **Public (público)**: os atributos podem ser acessados livremente em qualquer ponto do código.

Em Python, porém, não temos as palavras reservadas ```private```, ```protected``` ou ```public```. Ao invés disso, utilizaremos _underline_ (o símbolo ```_```) no início de nossos atributos:

```
self.atributoA = 'teste público'    # sem underline: atributo público
self._atributoB = 'teste protegido' # 1 underline: atributo protegido
self.__atributoC = 'teste privado'  # 2 underlines: atributo privado
```

Vejamos um exemplo baseado na nossa antiga classe Televisor:

In [ ]:
class Televisor:
    def __init__(self, marca, modelo, volume, canal):
        self.marca = marca     # público
        self._modelo = modelo  # protegido
        self.__volume = volume # privado
        self.canal = canal 
        
# Vamos aos testes:
televisor = Televisor('Samsung', 'UHD55SMART', 98, 'HBO')
print(televisor.marca)
print(televisor._modelo) 
print(televisor.__volume)


Samsung
UHD55SMART


AttributeError: 'Televisor' object has no attribute '__volume'

Como era de se esperar, ao tentarmos acessar o atributo ```__volume```, nos deparamos com um erro. Estávamos fora da classe e tentamos acessar um atributo privado. O Python age como se o atributo não existisse, afinal, fora da classe ninguém deveria mexer com ele.

A surpresa está no atributo ```_modelo```. Ele não deveria ser protegido? Sim. Porém, o Python possui uma política chamada "_we're all consenting adults_". Isso significa que não existe qualquer grau **real** de privacidade nessa linguagem. Se nós realmente quisermos e soubermos o que estamos fazendo, podemos acessar e modificar qualquer tipo de atributo, inclusive os privados:

* A sintaxe de argumento protegido (1 _underline_) é apenas uma **convenção**. Ou seja, ela não possui efeito prático no código. Porém, outros programadores saberão que ela não deveria ser modificada fora da "família" da classe ao ver o símbolo. Da mesma forma, alguns _debuggers_ e _linters_ poderão alertar o desenvolvedor de que ele está usando de maneira indevida aquele atributo.

* A sintaxe de argumento privado (2 _underlines_) de fato esconde o argumento. Porém, ela o faz utilizando uma estratégia chamada de "_name mangling_" (algo como "deturpação de nome"). Isso significa que ela "bagunça" o nome do atributo, de modo que o nome "oficial" não vale mais. Porém, a estratégia utilizada para o _mangling_ é bem conhecida:

```
_NomeDaClasse__NomeDoAtributo
```

Observe o que acontece quando utilizamos esse padrão para acessar um dos atributos privados do Televisor:

In [ ]:
print(televisor._Televisor__volume)

## Métodos de acesso
Supondo que todos os programadores sejam aderentes às boas práticas de programação e não pretendam explorar a política de _consenting adults_ do Python para bagunçar nossos atributos, como faremos para permitir que eles possam ler ou mesmo alterar (respeitando as regras de negócio) nossos atributos?

Aqui entram os métodos de acesso. Programadores de diversas linguagens orientadas a objeto os usam. O padrão é sempre mantermos nossos atributos privados e criar 2 métodos para cada um deles, um conhecido como _getter_ e outro como _setter_. Eles funcionam da seguinte maneira:

* O **getter** normalmente recebe o nome getNomeDoAtributo, não possui parâmetros e simplesmente retorna o valor atual do atributo. 
* O **setter** normalmente recebe o nome setNomeDoAtributo e recebe o novo valor como parâmetro. Ele deve **validar** o valor de acordo com as regras da classe, e caso o valor seja válido, o _setter_ irá copiá-lo para o atributo. _Setters_ tipicamente não retornam nada.

Sendo assim, vamos mais uma vez atualizar nosso Televisor:

In [ ]:
class Televisor:
    # Note que podemos usar get/set até mesmo no construtor! 
    # Isso vai deixar nossa classe mais segura, evitando que objetos sejam CRIADOS com valores indevidos:
    def __init__(self, marca, modelo, volume, canal, lista_canais):
        self.setListaCanais(lista_canais)
        self.setMarca(marca) 
        self.setModelo(modelo)
        self.setVolume(volume)
        self.setCanal(canal)        
        
    def getMarca(self):
        return self.__marca
    def setMarca(self, marca):
        self.__marca = marca
        
    def getModelo(self):
        return self.__modelo
    def setModelo(self, modelo):
        self.__modelo = modelo
        
    def getVolume(self):
        return self.__volume
    def setVolume(self, volume):
        if volume > 100:
            self.__volume = 100
        elif volume < 0:
            self.__volume = 0
        else:
            self.__volume = volume
        
    def getCanal(self):
        return self.__canal
    def setCanal(self, canal):
        if canal in self.getListaCanais(): # note que podemos fazer get de uma atributo mesmo dentro da classe!
            self.__canal = canal
        elif len(self.getListaCanais()) > 0:
            self.__canal = self.getListaCanais()[0]
        else:
            self.__canal = None
            
    def getListaCanais(self):
        return self.__lista_canais
    def setListaCanais(self, lista):
        if type(lista) == list:
            self.__lista_canais = lista
        else:
            self.__lista_canais = []

In [ ]:
televisor = Televisor('Samsung', 'UHD55SMART', 98, 'HBO', ['Globo', 'SBT', 'Manchete'])

# note que tentamos colocar lá em cima um canal inexistente (HBO)
# porém, o construtor usou os setters para definir os atributos
# o setter de canal prevê o seguinte comportamento: se o canal não está na lista, pega o primeiro da lista
# logo, qual será o canal sintonizado no momento?

#print(televisor.canal)

print(televisor.getCanal()) 



Globo


Note que agora mesmo nem sequer tenhamos implementado ainda um método de aumentar/diminuir volume na classe Televisor, qualquer outra classe que queira mexer no volume terá que passar pelo setter do volume, e este _setter_ já evita valores superiores a 100 ou inferiores a 0. 

Portanto, nosso amigo programador ~~porco~~ distraído que irá implementar a classe ControleRemoto não precisa fazer verificações adicionais e ainda assim nosso objeto está seguro:

In [ ]:
class ControleRemoto:
    def __init__(self, tv):
        self.tv = tv
    ...
    def aumentar_volume(self):
        atual = self.tv.getVolume()
        self.tv.setVolume(atual+1)

controle = ControleRemoto(televisor)

# Aumentando via controle:
controle.aumentar_volume() # Funciona, foi pra 99
print(televisor.getVolume()) # note que sempre que queremos visualizar, utilizamos o getter

controle.aumentar_volume() # Funciona, foi pra 100
print(televisor.getVolume())

controle.aumentar_volume() # Não funciona, fica em 100
print(televisor.getVolume())

AttributeError: 'Televisor' object has no attribute 'volume'

## _Getters_ e _setters_ "pythonistas"

As vantagens de utilizar _getters_ e _setters_ tradicionais estão evidentes. O encapsulamento da classe Televisor está muito mais robusto. Seus atributos estão bem protegidos, e mesmo programadores pouco cuidadosos dificilmente conseguirão colocar um objeto Televisor em um estado inválido (por exemplo, com volume = 101).

Porém, houve um custo: a sintaxe ficou mais carregada. Trocamos código intuitivo por código um pouco menos intuitivo e mais carregado:

```tv.volume = 10``` virou ```tv.setVolume(10)```

```vol = tv.volume``` virou ```vol = tv.getVolume()```

Felizmente, o Python fornece meios para combinarmos o melhor dos dois mundos: a sintaxe limpa do acesso direto aos atributos com a segurança da checagem de valores através de métodos. Temos duas formas de fazer isso, e abordaremos ambas.

### 1ª forma: _@property_ + _@atributo.setter_

Para nosso novo código, utilizaremos algo que ainda não apareceu. Acima de alguns de nossos métodos, colocaremos algumas palavrinhas precedidas por ```@```. Isso é o que chamamos de ```decorator```, e serve para sinalizar para o Python que um certo método possui algumas propriedades especiais.

O _decorator_ ```@property``` sinaliza para o Python que o método abaixo será o _getter_ de algum atributo. Já o ```@atributo.setter``` indica que o método abaixo servirá como _setter_  para o atributo chamado "atributo".

Qual a vantagem deste método? Nós, que estamos desenvolvendo a classe, iremos enxergá-los como métodos. Porém, quem utiliza a classe, irá enxergá-los como um atributo. Ou seja, eles poderão ler o atributo diretamente:

```
valor = objeto.atributo
print(objeto.atributo)
# etc

objeto.atributo = 'novo valor'
```

Porém, ele não estará acessando o atributo. O Python irá substituir esses supostos acessos ao atributo por chamadas para os métodos automaticamente. 

Vamos mais uma ver manipular nosso Televisor (ou uma versão simplificada dele):

In [ ]:
class Televisor:
    def __init__(self, volume):
        # IMPORTANTE:
        # não existe atributo volume!
        # aqui estamos implicitamente chamando o setter definido lá embaixo!
        # é dentro do próprio setter que o atributo __volume será criado ao receber um valor pela primeira vez
        self.volume = volume
    
    # getter para volume
    # note que utilizamos o nome que gostaríamos que aparecesse como atributo para os outros programadores (volume)
    @property
    def volume(self):
        return self.__volume
    
    # setter para volume
    # note que utilizamos o nome que gostaríamos que aparecesse como atributo para os outros programadores
    @volume.setter
    def volume(self, valor): 
        if valor > 100:
            self.__volume = 100
        elif valor < 0:
            self.__volume = 0
        else:
            self.__volume = valor

In [ ]:
tv = Televisor(102) # tentando passar um volume inválido...
print(tv.volume) # note qual valor foi parar lá...

tv.volume = 10 # passando um volume "bem comportado"
print(tv.volume) #... e funciona!

tv.volume = -5 # outro volume estranho...
print(tv.volume) # e o setter nos salvou de novo


tv.volume += 1

tv.volume = tv.volume + 1
print(tv.volume)

100
10
0
2


In [ ]:
class ControleRemoto:
    def __init__(self, tv):
        self.tv = tv
    def aumentar_volume(self):
        self.tv.volume += 1

### 2ª forma: função _property_

Como pudemos observar nos exemplos acima, o uso dos _decorators_ melhorou significativamente nosso código. O excesso de "get" e "set" para todos os lados deixava nosso código verboso e com mais carinha de Java do que de Python. Ao aplicarmos os _decorators_, voltamos a ter carinha de Python.

Porém, há quem ache burocrático escrever diversas arrobas dentro da classe. Felizmente, existe uma segunda estratégia:

* 1. Criar métodos get e set sem os decorators.
* 2. (Opcional) Tornar esses métodos privados, para não confundir outros programadores.
* 3. Criar um "atalho" passando esses métodos para a função property().

Vamos mexer com a nossa classe Televisor pela última vez:

In [ ]:
class Televisor:
    def __init__(self, volume):
        self.volume = volume
        
    # Note o __ no início. Esses métodos são privados. Isso é opcional.
    def __get_volume(self):
        return self.__volume
    
    def __set_volume(self, valor):        
        if valor > 100:
            self.__volume = 100
        elif valor < 0:
            self.__volume = 0
        else:
            self.__volume = valor
    
    # Tendo o getter e o setter, criamos a propriedade:
    volume = property(__get_volume, __set_volume)
    # Note que passamos as funções SEM parênteses
    # Não estamos chamando a função para passar seus retornos. Estamos passando as funções em si.

Note que a classe acima se comportará EXATAMENTE como a da primeira forma. Inclusive iremos repetir os mesmos exemplos e chegar nos mesmos resultados:

In [ ]:
tv = Televisor(102) # tentando passar um volume inválido...
print(tv.volume) # note qual valor foi parar lá...

tv.volume = 10 # passando um volume "bem comportado"
print(tv.volume) #... e funciona!

tv.volume = -5 # outro volume estranho...
print(tv.volume) # e o setter nos salvou de novo

100
10
0


### Observação: _decorators_

Isso não pertence ao escopo dessa aula, e você pode seguir para os exercícios se quiser. Porém, fizemos uma simplificação ao explicar o que é _decorator_, e aqui está uma explicação mais completa sobre o que eles são.

Lá em cima, definimos _decorators_ como um jeitinho de marcar uma função como possuindo algum tipo de propriedade especial.

Na realidade, _decorators_ tem mais a ver com a passagem de funções como parâmetros para outras funções. Veja o exemplo abaixo:

In [ ]:
def funcao_externa(funcao_parametro):
    # Note que a função interna possui acesso ao parâmetro da função externa
    def funcao_interna():
        print('Olá')
        funcao_parametro()
        print('Mundo')
    return funcao_interna()

# Criando uma função nova e "decorando" com a funcao_externa:
@funcao_externa
def teste():
    print('teste')

Note que nem sequer precisamos **chamar** a função teste. Ao declará-la com um _decorator_, automaticamente a função que recebe o nome do _decorator_ foi chamada passando a função "decorada" como parâmetro. Especificamente em nosso exemplo, a função decorada acaba chamando a função passada, mas isso não é necessário.

A explicação ainda simplificada, mas factualmente correta para o uso dos _decorators_ para criar _getters_ e _setters_ é a seguinte: nós estamos criando a lógica de cada _getter_ e _setter_, e ao decorá-las, estamos passando essa lógica (nossas funções) para uma outra função que irá criar os _getters_/_setters_ (a tal da _property_) padrões do Python para nós de maneira automática.

# Exercícios:

Utilize boas práticas de encapsulamento (atributos privados e properties/getters e setters) em **todos** os atributos das classes dos exercícios da aula 01.